In [14]:
import os
import glob
import pandas as pd
import numpy as np
import re

import math
import matplotlib.pyplot as plt
import matplotlib as mpl

np.Inf = np.inf


In [3]:
# ─── 1) POINT TO YOUR DIRECTORY ───────────────────────────────────────────────
results_dir = "/home/sagnik/Research/Hierarchical_AMP/Results/vary_rho_com_meth"
pattern = os.path.join(results_dir, "rho_*.csv")

# ─── 2) READ & COMBINE FILES ───────────────────────────────────────────────────
records = []
for fn in sorted(glob.glob(pattern)):
    basename = os.path.basename(fn)
    match = re.match(r"rho_(?P<rho>[\d.]+)_n_(?P<n>\d+)\.csv", basename)
    if not match:
        continue
    rho = float(match.group("rho"))
    n = int(match.group("n"))

    df = pd.read_csv(fn)
    df["rho"] = rho
    df["n"] = n
    df["log_rec_error"] = np.log(df["Error"])
    records.append(df[["Modality", "rho", "n", "Method", "log_rec_error"]])

# ─── 3) CONCATENATE ────────────────────────────────────────────────────────────
full_df = pd.concat(records, ignore_index=True)

# ─── 4) (Optional) SAVE ────────────────────────────────────────────────────────
out_path = os.path.join(results_dir, "combined_rec_error_log.csv")
full_df.to_csv(out_path, index=False)
print("✅ Combined DataFrame saved to:", out_path)

✅ Combined DataFrame saved to: /home/sagnik/Research/Hierarchical_AMP/Results/vary_rho_com_meth/combined_rec_error_log.csv


In [ ]:
# ─── 0) Path to the combined CSV ───────────────────────────────────────────────
results_dir = "/home/sagnik/Research/Hierarchical_AMP/Results/vary_rho_com_meth"
csv_path    = os.path.join(results_dir, "combined_rec_error_log.csv")

# ─── 1) Plot styling ───────────────────────────────────────────────────────────
mpl.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.serif": ["Times New Roman", "DejaVu Serif"],
    "axes.titlesize": 14,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 12,
    "lines.linewidth": 2,
    "lines.markersize": 6,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.6
})

# ─── 2) Load CSV and prepare plot output dir ───────────────────────────────────
df = pd.read_csv(csv_path)
plot_dir = os.path.join(results_dir, "Plots_rho_vs_n")
os.makedirs(plot_dir, exist_ok=True)

methods    = sorted(df["Method"].unique())
modalities = sorted(df["Modality"].unique())
rhos       = sorted(df["rho"].unique())

# ─── 3) Generate plots ─────────────────────────────────────────────────────────
for mod in modalities:
    for rho in rhos:
        sub = df[(df["Modality"] == mod) & (df["rho"] == rho)]

        if sub.empty:
            continue

        plt.figure(figsize=(6.5, 4.2))

        for m in methods:
            dpm = sub[sub["Method"] == m].sort_values("n")
            if dpm.empty:
                continue
            plt.plot(
                dpm["n"].to_numpy(),
                dpm["log_rec_error"].to_numpy(),
                marker='o',
                label=m
            )

        plt.xlabel("Sample size $n$")
        plt.ylabel(r"$\log\left(\frac{1}{n^2} \|\widehat{U}\widehat{U}^\top - UU^\top\|_F^2\right)$")
        plt.title(f"Modality {mod}, $\\rho={rho}$")
        plt.legend(loc="best", frameon=False)
        plt.tight_layout()

        fname = f"log_rec_vs_n_mod{mod}_rho{rho}.png"
        plt.savefig(os.path.join(plot_dir, fname), dpi=150)
        plt.close()

print("✅ Plots saved to:", plot_dir)

     Modality   rho     n     Method  log_rec_error
0           1  0.85  2000  clustered      -4.054714
1           2  0.85  2000  clustered      -3.879556
2           3  0.85  2000  clustered      -6.354332
3           1  0.85  2000   distinct      -3.236876
4           2  0.85  2000   distinct      -3.088439
..        ...   ...   ...        ...            ...
595         2  1.00  4000        mfa      -1.231693
596         3  1.00  4000        mfa      -1.867125
597         1  1.00  4000       hpca      -1.251087
598         2  1.00  4000       hpca      -0.998090
599         3  1.00  4000       hpca      -1.867368

[600 rows x 5 columns]
✅ Plots saved to: /home/sagnik/Research/Hierarchical_AMP/Results/vary_rho_com_meth/Plots_rho_vs_n


In [16]:
# ─── 0) Path to your combined CSV ───────────────────────────────────────────────
results_dir = "/home/sagnik/Research/Hierarchical_AMP/Results/vary_rho_com_meth"
csv_path    = os.path.join(results_dir, "combined_rec_error_log.csv")

# ─── 1) Plot styling ────────────────────────────────────────────────────────────
plt.rcParams.update({
    "text.usetex": False,
    "font.family": "serif",
    "font.serif": ["Times New Roman","DejaVu Serif"],
    "axes.titlesize": 14,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12,
    "legend.fontsize": 10,
    "lines.linewidth": 2,
    "lines.markersize": 6,
    "axes.grid": True,
    "grid.linestyle": "--",
    "grid.alpha": 0.6,
})

# ─── 2) Load data & prep output dir ─────────────────────────────────────────────
df       = pd.read_csv(csv_path)
plot_dir = os.path.join(results_dir, "PanelPlots_rho_vs_n")
os.makedirs(plot_dir, exist_ok=True)

methods    = sorted(df["Method"].unique())
modalities = sorted(df["Modality"].unique())
rhos       = sorted(df["rho"].unique())

# ─── 3) For each modality, build a multi‐subplot panel ──────────────────────────
for mod in modalities:
    n_rho = len(rhos)
    ncols = 2
    nrows = math.ceil(n_rho / ncols)
    fig, axes = plt.subplots(nrows, ncols,
                             figsize=(6.5 * ncols, 4.2 * nrows),
                             sharex=True, sharey=True,
                             constrained_layout=True)
    axes_flat = axes.flatten()

    for ax, rho in zip(axes_flat, rhos):
        sub = df[(df["Modality"] == mod) & (df["rho"] == rho)]
        if sub.empty:
            ax.axis("off")
            continue

        for m in methods:
            dpm = sub[sub["Method"] == m].sort_values("n")
            if dpm.empty:
                continue
            ax.plot(
                dpm["n"].to_numpy(),
                dpm["log_rec_error"].to_numpy(),
                marker="o",
                label=m
            )

        ax.set_title(rf"$\rho={rho}$")
        ax.set_xlabel("Sample size $n$")
        ax.set_ylabel(r"$\log\left(\frac{1}{n^2}\|\widehat U\widehat U^\top - U U^\top\|_F^2\right)$")

    # hide any extra subplots
    for ax in axes_flat[n_rho:]:
        ax.axis("off")

    # shared legend across the figure
    handles, labels = axes_flat[0].get_legend_handles_labels()
    fig.legend(handles, labels,
               loc="upper center",
               ncol=len(methods),
               bbox_to_anchor=(0.5, 1.02),
               frameon=False)

    fig.suptitle(f"Modality {mod}", y=1.03)
    out_fname = f"panel_modality_{mod}.png"
    fig.savefig(os.path.join(plot_dir, out_fname),
                dpi=150,
                bbox_inches="tight")
    plt.close(fig)

print("✅ Panel plots saved in", plot_dir)

✅ Panel plots saved in /home/sagnik/Research/Hierarchical_AMP/Results/vary_rho_com_meth/PanelPlots_rho_vs_n
